In [27]:
import pandas as pd
import numpy as np

df = pd.read_csv("Arrival data.csv")

In [30]:
df = df.iloc[:,:4]
df

,Week,Day,Arrival,Adjusted Arrival
0,1,Weekday,08:01:26,00:01:26
1,1,Weekday,08:06:59,00:06:59
2,1,Weekday,08:11:07,00:11:07
3,1,Weekday,08:13:08,00:13:08
4,1,Weekday,08:13:56,00:13:56
...,...,...,...,...
42066,8,Sun,00:52:20,16:52:20
42067,8,Sun,00:53:12,16:53:12
42068,8,Sun,00:55:38,16:55:38
42069,8,Sun,00:58:41,16:58:41


In [39]:
weekday = df[df['Day']=='Weekday']
sat = df[df['Day']=='Sat']
sun = df[df['Day']=='Sun']

In [40]:
sun

,Week,Day,Arrival,Adjusted Arrival
28187,1,Sun,08:03:00,00:03:00
28188,1,Sun,08:05:32,00:05:32
28189,1,Sun,08:05:45,00:05:45
28190,1,Sun,08:06:55,00:06:55
28191,1,Sun,08:08:02,00:08:02
...,...,...,...,...
42066,8,Sun,00:52:20,16:52:20
42067,8,Sun,00:53:12,16:53:12
42068,8,Sun,00:55:38,16:55:38
42069,8,Sun,00:58:41,16:58:41
